# Import libraries

In [ ]:
import jax
import ximinf.nn_inference as nninf
import ximinf.generate_sim as gsim
import pandas as pd
import jax.numpy as jnp

# Set device type

In [ ]:
jax.devices()
cpu = jax.devices("cpu")[0]
gpu = jax.devices("gpu")[0]

# Load data

In [ ]:
columns = columns = ['magobs', 'magobs_err','x1', 'x1_err', 'c', 'c_err', 'prompt', 'localcolor', 'localcolor_err', 'z']
df = pd.read_parquet("../data/inference_data_frame.parquet")
flat = gsim.flatten_df(df, columns)

# Load NN

In [ ]:
# Parameters
N = 1  # Number of samples # RETRIEVE FROM DATA
M = 100      # Number of points per sample # RETRIEVE FROM DATA

model = nninf.load_nn('../data/NNs/nn_model')

# MCMC

In [ ]:
model.eval()

# ========== Global Settings ==========
#BOUNDS = jnp.array([[0., 1.], [0., 1.]]) #normalised
BOUNDS = jnp.array([[-1, 1], [2, 4]])
NDIM = BOUNDS.shape[0]

In [ ]:
print("Launch MCMC ...")

interleaved = jnp.stack([
    df['magobs'].squeeze(),
    df['x1'].squeeze(),
    df['c'].squeeze(),
], axis=1).reshape(-1)  # Shape (2*M,)

# Append errors after interleaved points
interleaved = jnp.concatenate([
    interleaved,
    mask,
    jnp.array([err_x, err_y])
])  # Shape (2*M + 2,)

def log_post(theta):
        return nninf.log_prob_fn(theta, model, interleaved)

# Run MCMC
key, post = nninf.sample_posterior(
    log_post,
    n_warmup=200,
    n_samples=2000,
    init_position = theta_true,
    rng_key=key
)

print("...finished")